# FileSystem (Polyglot)

In [ ]:
#!import ../nbs/Async.dib

# Async (Polyglot)

# Common (Polyglot)

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

...

Installing Packages FSharp.Control.AsyncSeq..... System.Reactive..... System.Reactive.Linq.....

Installing Packages FSharp.Control.AsyncSeq...... System.Reactive...... System.Reactive.Linq......

Installing Packages FSharp.Control.AsyncSeq....... System.Reactive....... System.Reactive.Linq.......

Installed Packages FSharp.Control.AsyncSeq, 3.2.1 System.Reactive, 5.0.0 System.Reactive.Linq, 6.0.0-preview.1

00010101-0000-0000-0000-0a9876543210
99991231-2359-5999-9999-9a9876543210
19700101-0000-0000-0000-0a9876543210
0001-01-01 00:00:00Z
9999-12-31 23:59:59Z
1970-01-01 00:00:00Z
00000000-0000-0000-00dc-ba9876543210
99999999-9999-9999-99dc-ba9876543210
0
999999999999999999
0
0
13:32:18 #1 [Debug] test
1
1
1
0
13:32:22 #1 [Debug] runWithTimeout / timeout: 10 / exception: The operation has timed out.
<null>
FSharpOption<Unit>
      Value: <null>
13:32:23 #2 [Debug] runWithTimeout / timeout: 100 / exception: The operation has timed out.
<null>
[ 1, AddHandler, 2, RemoveHandler ]
13:32:23 #3 [Debug] runWithTimeout / timeout: 100 / exception: The operation has timed out.
<null>
[ 1, AddHandler, 2, IObservable.Subscribe, 3, RemoveHandler, IObservable.Dispose ]
13:32:23 #4 [Debug] runWithTimeout / timeout: 100 / exception: The operation has timed out.
<null>
[ 1, AddHandler, 2, IObservable.Subscribe, 3, TestEvent.Subscribe(error), TestEvent.Iter(0:error), testEvent.EventInterface.Subscribe(error),

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs

In [ ]:
open Common
open Async

## TempDirectory

In [ ]:
let createTempDirectoryName () =
    let root =
        match System.Reflection.Assembly.GetEntryAssembly().GetName().Name with
        | assemblyName -> assemblyName

    System.IO.Path.GetTempPath ()
    </> $"!{root}"
    </> string (newGuidFromDateTime System.DateTime.Now)

let createTempDirectory () =
    let tempFolder = createTempDirectoryName ()
    let result = System.IO.Directory.CreateDirectory tempFolder

    let getLocals () =
        $"tempFolder: {tempFolder} / result: {({|
            Exists = result.Exists
            CreationTime = result.CreationTime
        |})} {getLocals ()}"

    trace Debug (fun () -> "createTempDirectory") getLocals

    tempFolder

In [ ]:
//// test

createTempDirectoryName ()

C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2392-9218-9652feb7c0a4

## WaitForFileAccess

In [ ]:
let rec waitForFileAccess path = async {
    let rec loop retry = async {
        try
            use _ = new FileStream (path, FileMode.Open, FileAccess.ReadWrite)
            ()
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / message: {ex.Message} / {getLocals ()}"
                trace Warn (fun () -> nameof waitForFileAccess) getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    return! loop 0
}

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use stream = new FileStream (path, FileMode.Open, FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
    stream.Seek (0L, SeekOrigin.Begin) |> ignore
    trace Debug (fun () -> "_4") getLocals
    stream.WriteByte (49uy) |> ignore
    trace Debug (fun () -> "_5") getLocals
    stream.Flush ()
    trace Debug (fun () -> "_6") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! _ = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    do! waitForFileAccess path
    trace Debug (fun () -> "5") getLocals
    let text = File.ReadAllText path
    trace Debug (fun () -> $"text: {text}") getLocals
    return text
}
|> Async.runWithTimeout 3000
|> _equal (Some "1")

13:32:24 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2420-2073-285ab53ca2d0 / result: { CreationTime = 2023-07-19 1:32:24 PM
  Exists = true }
13:32:24 #2 [Debug] 1
13:32:24 #3 [Debug] 2
13:32:24 #4 [Debug] 3
13:32:24 #5 [Debug] _1
13:32:24 #6 [Debug] _2
13:32:24 #7 [Debug] 4
13:32:24 #8 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2420-2073-285ab53ca2d0\test.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2420-2073-285ab53ca2d0\test.txt' because it is being used by another process.
13:32:25 #9 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2420-2073-285ab53ca2d0\test.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2420-2073-285ab53ca2d0\test.txt' because it is being used by another process.
13:32:26 #10 

## DeleteDirectoryAsync

In [ ]:
let rec deleteDirectoryAsync path = async {
    let rec loop retry = async {
        try
            System.IO.Directory.Delete (path, true)
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / message: {ex.Message} / {getLocals ()}"
                trace Warn (fun () -> nameof deleteDirectoryAsync) getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    return! loop 0
}

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test"

let lockDirectory () = async {
    trace Debug (fun () -> "_1") getLocals
    System.IO.File.WriteAllText (path </> "test.txt", "")
    use _ = new FileStream (path </> "test.txt", FileMode.Open, FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    Directory.CreateDirectory path |> ignore
    trace Debug (fun () -> "2") getLocals
    let! _ = lockDirectory () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    do! deleteDirectoryAsync path
    trace Debug (fun () -> "5") getLocals
}
|> Async.runWithTimeout 3000
|> _equal (Some ())

13:32:26 #16 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2651-5153-5d94146758d4 / result: { CreationTime = 2023-07-19 1:32:26 PM
  Exists = true }
13:32:26 #17 [Debug] 1
13:32:26 #18 [Debug] 2
13:32:26 #19 [Debug] 3
13:32:26 #20 [Debug] _1
13:32:26 #21 [Debug] _2
13:32:26 #22 [Debug] 4
13:32:26 #23 [Warn] deleteDirectoryAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2651-5153-5d94146758d4\test / message: The process cannot access the file 'test.txt' because it is being used by another process.
13:32:27 #24 [Warn] deleteDirectoryAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2651-5153-5d94146758d4\test / message: The process cannot access the file 'test.txt' because it is being used by another process.
13:32:28 #25 [Debug] _3
13:32:28 #26 [Debug] 5
FSharpOption<Unit>
      Value: <null>


## DeleteFileAsync

In [ ]:
let rec deleteFileAsync path = async {
    let rec loop retry = async {
        try
            System.IO.File.Delete path
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / message: {ex.Message} / {getLocals ()}"
                trace Warn (fun () -> nameof deleteFileAsync) getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    return! loop 0
}

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new FileStream (path, FileMode.Open, FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! _ = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    do! deleteFileAsync path
    trace Debug (fun () -> "5") getLocals
}
|> Async.runWithTimeout 3000
|> _equal (Some ())

13:32:28 #27 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2886-8667-8dfa5bd0973f / result: { CreationTime = 2023-07-19 1:32:28 PM
  Exists = true }
13:32:28 #28 [Debug] 1
13:32:28 #29 [Debug] 2
13:32:28 #30 [Debug] 3
13:32:28 #31 [Debug] _1
13:32:28 #32 [Debug] _2
13:32:28 #33 [Debug] 4
13:32:28 #34 [Warn] deleteFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2886-8667-8dfa5bd0973f\test.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2886-8667-8dfa5bd0973f\test.txt' because it is being used by another process.
13:32:30 #35 [Warn] deleteFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2886-8667-8dfa5bd0973f\test.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2886-8667-8dfa5bd0973f\test.txt' because it is being used by another process.
13:32:30

## MoveFileAsync

In [ ]:
let rec moveFileAsync newPath oldPath = async {
    let rec loop retry = async {
        try
            System.IO.File.Move (oldPath, newPath)
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / message: {ex.Message} / {getLocals ()}"
                trace Warn (fun () -> nameof moveFileAsync) getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    return! loop 0
}

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"
let newPath = tempFolder </> "test2.txt"

let lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new FileStream (path, FileMode.Open, FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! _ = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    do! path |> moveFileAsync newPath
    trace Debug (fun () -> "5") getLocals
    return File.ReadAllText newPath
}
|> Async.runWithTimeout 3000
|> _equal (Some "0")

13:32:31 #38 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3117-1793-17a1d42bbd64 / result: { CreationTime = 2023-07-19 1:32:31 PM
  Exists = true }
13:32:31 #39 [Debug] 1
13:32:31 #40 [Debug] 2
13:32:31 #41 [Debug] 3
13:32:31 #42 [Debug] _1
13:32:31 #43 [Debug] _2
13:32:31 #44 [Debug] 4
13:32:31 #45 [Warn] moveFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2886-8667-8dfa5bd0973f\test.txt / message: The process cannot access the file because it is being used by another process.
13:32:32 #46 [Warn] moveFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-2886-8667-8dfa5bd0973f\test.txt / message: The process cannot access the file because it is being used by another process.
13:32:33 #47 [Debug] _3
13:32:33 #48 [Debug] 5
FSharpOption<String>
      Value: 0


## FileSystemWatcher

In [ ]:
[<RequireQualifiedAccess>]
type FileSystemChangeType =
    | Error
    | Changed
    | Created
    | Deleted
    | Renamed

[<RequireQualifiedAccess>]
type FileSystemChange =
    | Error of exn: exn
    | Changed of path: string * content: string option
    | Created of path: string * content: string option
    | Deleted of path: string
    | Renamed of oldPath: string * (string * string option)


let watchWithFilter filter shouldReadContent path =
    let fullPath = System.IO.Path.GetFullPath path
    let getLocals () = $"fullPath: {fullPath} / filter: {filter} / {getLocals ()}"

    let watcher =
        new System.IO.FileSystemWatcher (
            Path = fullPath,
            NotifyFilter = filter,
            EnableRaisingEvents = true,
            IncludeSubdirectories = true
        )

    let getEventPath (path : string) = path.Trim().Replace(fullPath, "").TrimStart [| '/'; '\\' |]

    let ticks () = System.DateTime.UtcNow.Ticks

    let readContent fullPath =
        if not shouldReadContent
        then None
        else
            try
                waitForFileAccess fullPath |> Async.runWithTimeout 30000 |> ignore
                System.IO.File.ReadAllText fullPath |> Some
            with ex ->
                trace Error (fun () -> $"Failed to read file content: {ex.Message}") getLocals
                None

    let changedStream =
        AsyncSeq.subscribeEvent
            watcher.Changed
            (fun event ->
                ticks (),
                [ FileSystemChange.Changed (getEventPath event.FullPath, readContent event.FullPath) ]
            )

    let deletedStream =
        AsyncSeq.subscribeEvent
            watcher.Deleted
            (fun event ->
                ticks (),
                [ FileSystemChange.Deleted (getEventPath event.FullPath) ]
            )

    let createdStream =
        AsyncSeq.subscribeEvent
            watcher.Created
            (fun event ->
                let path = getEventPath event.FullPath
                let content = readContent event.FullPath
                ticks (), [
                    FileSystemChange.Created (path, content)
                    if OS.isWindows () then
                        FileSystemChange.Changed (path, content)
                ])

    let renamedStream =
        AsyncSeq.subscribeEvent
            watcher.Renamed
            (fun event ->
                ticks (), [
                    FileSystemChange.Renamed (
                        getEventPath event.OldFullPath,
                        (getEventPath event.FullPath, readContent event.FullPath)
                    )
                ]
            )

    let errorStream =
        AsyncSeq.subscribeEvent
            watcher.Error
            (fun event -> ticks (), [ FileSystemChange.Error (event.GetException ()) ])

    let stream =
        [
            changedStream
            deletedStream
            createdStream
            renamedStream
            errorStream
        ]
        |> FSharp.Control.AsyncSeq.mergeAll
        |> FSharp.Control.AsyncSeq.map (fun (n, events) ->
            events
            |> List.fold
                (fun (i, events) event ->
                    i + 1L,
                    (n + i, event) :: events)
                (0L, [])
            |> snd
            |> List.rev
        )
        |> FSharp.Control.AsyncSeq.concatSeq

    let disposable =
        Object.newDisposable (fun () ->
            trace Debug (fun () -> "Disposing watch stream") getLocals
            watcher.EnableRaisingEvents <- false
            watcher.Dispose ()
        )

    stream, disposable

let watch path =
    watchWithFilter
        (System.IO.NotifyFilters.Attributes
        ||| System.IO.NotifyFilters.CreationTime
        ||| System.IO.NotifyFilters.DirectoryName
        ||| System.IO.NotifyFilters.FileName
        //  ||| System.IO.NotifyFilters.LastAccess
        //  ||| System.IO.NotifyFilters.LastWrite
        ||| System.IO.NotifyFilters.Security
        //  ||| System.IO.NotifyFilters.Size
        )
        path

### testEventsRaw (test)

In [ ]:
//// test

let testEventsRaw (watchFn : bool -> string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable) write =
    let tempDirectory = createTempDirectory ()
    let stream, disposable = watchFn true tempDirectory

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! _ = iter () |> Async.StartChild
        do! Async.Sleep 150
        return! write tempDirectory
    }

    try
        run
        |> Async.runWithTimeout 10000
        |> _equal (Some ())
    finally
        disposable.Dispose ()
        deleteDirectoryAsync tempDirectory |> Async.RunSynchronously

    let eventsLog =
        events
        |> Seq.toList
        |> List.sortBy fst
        |> List.fold
            (fun (prev, acc) (ticks, event) -> ticks, (ticks, (if prev = 0L then 0L else ticks - prev), event) :: acc)
            (0L, [])
        |> snd
        |> List.rev
        |> List.map (fun (diff, n, event) ->
            let text = $"{n} / {diff} / {event}"
            text.Substring (0, min 100 text.Length)
        )
        |> String.concat "\n"
    let getLocals () = $"eventsLog: \n{eventsLog} / {getLocals ()}"
    trace Debug (fun () -> "testEventsRaw") getLocals

    events
    |> Seq.toList
    |> List.sortBy fst
    |> List.map snd
    |> List.fold
        (fun (map, acc) event ->
            match event with
            | FileSystemChange.Changed (path, (Some _ as content)) ->
                if map |> Map.containsKey path && map.[path] = content
                then map, acc
                else (map |> Map.add path content), event :: acc
            | FileSystemChange.Created (path, _)
            | FileSystemChange.Deleted path ->
                (map |> Map.remove path), event :: acc
            | FileSystemChange.Renamed (oldPath, (newPath, content)) ->
                (map |> Map.remove oldPath |> Map.remove newPath), event :: acc
            | _ -> map, event :: acc
        )
        (Map.empty, [])
    |> snd
    |> List.rev

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"a{i}") |> Async.AwaitTask

    do! Async.Sleep 30

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"b{i}") |> Async.AwaitTask

    do! Async.Sleep 30

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt")

    do! Async.Sleep 30

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"c{i}") |> Async.AwaitTask

    do! Async.Sleep 30

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt")
}

let events = testEventsRaw watch write

events
|> _sequenceEqual [
    FileSystemChange.Created ("file1.txt", Some "a1")
    FileSystemChange.Changed ("file1.txt", Some "a1")
    FileSystemChange.Created ("file2.txt", Some "a2")
    FileSystemChange.Changed ("file2.txt", Some "a2")

    FileSystemChange.Changed ("file1.txt", Some "b1")
    FileSystemChange.Changed ("file2.txt", Some "b2")

    FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "b1"))
    FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "b2"))

    FileSystemChange.Changed ("file_1.txt", Some "c1")
    FileSystemChange.Changed ("file_2.txt", Some "c2")

    FileSystemChange.Deleted "file_1.txt"
    FileSystemChange.Deleted "file_2.txt"
]

13:32:34 #49 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3427-2749-2427fd32d3ed / result: { CreationTime = 2023-07-19 1:32:34 PM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
13:32:34 #50 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3427-2749-2427fd32d3ed / filter: FileName, DirectoryName, Attributes, CreationTime, Security
13:32:34 #51 [Debug] testEventsRaw / eventsLog: 
0 / 638253811544554438 / Created ("file1.txt", Some "a1")
1 / 638253811544554439 / Changed ("file1.txt", Some "a1")
25627 / 638253811544580066 / Created ("file2.txt", Some "a2")
1 / 638253811544580067 / Changed ("file2.txt", Some "a2")
371532 / 638253811544951599 / Changed ("file1.txt", Some "b1")
5673 / 638253811544957272 / Changed ("file2.txt", Some "b2")
506621 / 638253811545463893 / Renamed ("file1.txt", ("file_1.txt", Some "b1"))
19545 / 638253811545483438 / Renamed ("file2.txt", ("file

### testEventsSorted (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    try
        let contents =
            [1..n]
            |> List.map (fun i -> String.replicate 1_000_000 (string i))

        for i = 1 to n do
            do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"a{contents.[i - 1]}") |> Async.AwaitTask

        do! Async.Sleep 30

        for i = 1 to n do
            do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"b{contents.[i - 1]}") |> Async.AwaitTask

        do! Async.Sleep 30

        for i = 1 to n do
            do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt")

        do! Async.Sleep 30

        for i = 1 to n do
            do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"c{contents.[i - 1]}") |> Async.AwaitTask

        do! Async.Sleep 30

        for i = 1 to n do
            do! deleteFileAsync (path </> $"file_{i}.txt")
    with ex ->
        trace Error (fun () -> $"write: {ex.Message}") getLocals
}

let events =
    testEventsRaw watch write
    |> List.map (function
        | FileSystemChange.Changed (path, Some content) ->
            FileSystemChange.Changed (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
        | FileSystemChange.Created (path, Some content) ->
            FileSystemChange.Created (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
        | FileSystemChange.Renamed (oldPath, (newPath, Some content)) ->
            FileSystemChange.Renamed (
                oldPath,
                (newPath, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
            )
        | event -> event
    )

events
|> _sequenceEqual [
    FileSystemChange.Created ("file1.txt", Some "a1")
    FileSystemChange.Changed ("file1.txt", Some "a1")
    FileSystemChange.Created ("file2.txt", Some "a2")
    FileSystemChange.Changed ("file2.txt", Some "a2")

    FileSystemChange.Changed ("file1.txt", Some "b1")
    FileSystemChange.Changed ("file2.txt", Some "b2")

    FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "b1"))
    FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "b2"))

    FileSystemChange.Changed ("file_1.txt", Some "c1")
    FileSystemChange.Changed ("file_2.txt", Some "c2")

    FileSystemChange.Deleted "file_1.txt"
    FileSystemChange.Deleted "file_2.txt"
]

13:32:35 #52 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3511-1118-14502bdb3f71 / result: { CreationTime = 2023-07-19 1:32:35 PM
  Exists = true }
13:32:35 #53 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3511-1118-14502bdb3f71\file1.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3511-1118-14502bdb3f71\file1.txt' because it is being used by another process.
13:32:35 #54 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3511-1118-14502bdb3f71\file2.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3511-1118-14502bdb3f71\file2.txt' because it is being used by another process.
13:32:35 #55 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3511-1118-14502bdb3f71\file1.txt 

In [ ]:
//// test

let sortEvent event =
    match event with
    | FileSystemChange.Error _ -> 0
    | FileSystemChange.Created _ -> 1
    | FileSystemChange.Changed _ -> 2
    | FileSystemChange.Renamed (_oldPath, _) -> 3
    | FileSystemChange.Deleted _ -> 4

let formatEvents events =
    events
    |> Seq.toList
    |> List.sortBy (snd >> sortEvent)
    |> List.choose (fun (ticks, event) ->
        match event with
        | FileSystemChange.Error _ ->
            None
        | FileSystemChange.Changed (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Changed)
        | FileSystemChange.Created (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Created)
        | FileSystemChange.Deleted path ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Deleted)
        | FileSystemChange.Renamed (_oldPath, (path, _)) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Renamed)
    )
    |> List.sortBy (fun (_, path, _) -> path)
    |> List.distinctBy (fun (_, path, event) -> path, event)

let testEventsSorted (watchFn : string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable) write =
    let path = createTempDirectory ()
    let stream, disposable = watchFn path

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! child = iter () |> Async.StartChild
        do! Async.Sleep 150
        do! write path
        do! child
    }

    try
        run
        |> Async.runWithTimeout 500
        |> ignore
    finally
        disposable.Dispose ()
        deleteDirectoryAsync path |> Async.RunSynchronously

    let events = formatEvents events

    let eventMap =
        events
        |> List.map (fun (ticks, path, event) -> path, (event, ticks))
        |> List.groupBy fst
        |> List.map (fun (path, events) ->
            let event, _ticks =
                events
                |> List.map snd
                |> List.sortByDescending snd
                |> List.head

            path, event
        )
        |> Map.ofList

    let eventList =
        events
        |> List.map (fun (_ticks, path, event) -> path, event)

    eventMap, eventList

#### create and delete (test)

In [ ]:
//// test

let write path = async {
    let n = 3

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt")
}

let eventMap, eventList = testEventsSorted (watch false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file1.txt", nameof FileSystemChangeType.Deleted

    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Deleted

    "file3.txt", nameof FileSystemChangeType.Created
    "file3.txt", nameof FileSystemChangeType.Changed
    "file3.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Deleted
    "file2.txt", nameof FileSystemChangeType.Deleted
    "file3.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

13:32:36 #60 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3665-6519-67506c107368 / result: { CreationTime = 2023-07-19 1:32:36 PM
  Exists = true }
13:32:37 #61 [Debug] runWithTimeout / timeout: 500 / exception: The operation has timed out.
13:32:37 #62 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3665-6519-67506c107368 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
( file3.txt, Created )
( file3.txt, Changed )
( file3.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted
      - Key: file3.txt
        Value: Deleted


#### change (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt")
}

let eventMap, eventList = testEventsSorted (watch false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file1.txt", nameof FileSystemChangeType.Deleted

    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Deleted
    "file2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

13:32:37 #63 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3744-4403-469314f307a7 / result: { CreationTime = 2023-07-19 1:32:37 PM
  Exists = true }
13:32:37 #64 [Debug] runWithTimeout / timeout: 500 / exception: The operation has timed out.
13:32:37 #65 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3744-4403-469314f307a7 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted


#### rename (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt")

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt")
}

let eventMap, eventList = testEventsSorted (watch false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed

    "file_1.txt", nameof FileSystemChangeType.Renamed
    "file_1.txt", nameof FileSystemChangeType.Deleted

    "file_2.txt", nameof FileSystemChangeType.Renamed
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Deleted
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

13:32:38 #66 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3814-1418-1da0d382b670 / result: { CreationTime = 2023-07-19 1:32:38 PM
  Exists = true }
13:32:38 #67 [Debug] runWithTimeout / timeout: 500 / exception: The operation has timed out.
13:32:38 #68 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3814-1418-1da0d382b670 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted


#### full (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "") |> Async.AwaitTask

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt")

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt")
}

let eventMap, eventList = testEventsSorted (watch false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed

    "file_1.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Renamed
    "file_1.txt", nameof FileSystemChangeType.Deleted

    "file_2.txt", nameof FileSystemChangeType.Changed
    "file_2.txt", nameof FileSystemChangeType.Renamed
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Deleted
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

13:32:39 #69 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3905-0524-04daac22c858 / result: { CreationTime = 2023-07-19 1:32:39 PM
  Exists = true }
13:32:39 #70 [Debug] runWithTimeout / timeout: 500 / exception: The operation has timed out.
13:32:39 #71 [Debug] Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230719-1332-3905-0524-04daac22c858 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Changed )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted
